In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fine-tuning Gemini to translate multiple languages to English using Vertex AI, Dataproc and Apache Iceberg

This notebooks demonstrates how to fine-tune Gemini to perform translation tasks from multiple languages (DE,ES,FR,IT,PL,PT,RU,SV,UK,ZH) to English.  
It follows the steps:  
  - Reads the dataset from a GCS bucket
  - Save it in the Iceberg format using Dataproc Serverless
  - Use Vertex AI Supervised fine-tuning job to fine-tune Gemini with the dataset
  - Register the model in Vertex AI Model Registry

<br>
<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/generative_ai/finetuning/translation_finetuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/generative_ai/finetuning/translation_finetuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/ai-ml-recipes/main/notebooks/generative_ai/finetuning/translation_finetuning.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/generative_ai/finetuning/translation_finetuning.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fai-ml-recipes%2Fmain%2Fnotebooks%2Fgenerative_ai%2Ffinetuning%2Ftranslation_finetuning.ipynb">
    <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
    Open in Colab Enterprise
    </a>
  </td>

</table>

#### Setup

In [ ]:
%pip install google-spark-connect -q

In [63]:
from typing import List
import pandas as pd
import time
import json

from google.cloud import bigquery, storage
from google.cloud.dataproc_v1 import Session, SparkConnectConfig
from google.cloud.spark_connect import GoogleSparkSession
from pyspark.sql.types import LongType, StringType, DoubleType

type_mapping = {
        LongType: 'long',
        StringType: 'string',
        DoubleType: 'double'
}

#### Config

In [ ]:
PROJECT_ID = "PROJECT_ID"
REGION = "REGION"
RUNTIME_TEMPLATE = "RUNTIME_TEMPLATE"

JSON_FILES_GCS_URI = "gs://dataproc-metastore-public-binaries/wikipedia_translated_clusters/*"
BUCKET_NAME = "BUCKET"

To run this notebook you must create a [Dataproc Serverless Runtime Template](https://cloud.google.com/dataproc-serverless/docs/quickstarts/jupyterlab-sessions) version 3.0

In [ ]:
session_config = Session()
session_config.spark_connect_session = SparkConnectConfig()
session_config.session_template = f'projects/{PROJECT_ID}/locations/{REGION}/sessionTemplates/{RUNTIME_TEMPLATE}'

spark = GoogleSparkSession.builder.projectId(PROJECT_ID).location(REGION).googleSessionConfig(session_config).getOrCreate()

Creating Spark session. It may take few minutes.
Interactive Session Detail View:  https://console.cloud.google.com/dataproc/interactive/us-east1/sc-20250416-190912-rf2ni8?project=dataproc-notebook-demo


#### Read input dataset

In [68]:
raw_dataset = spark.read.json(JSON_FILES_GCS_URI)

#### Transform the dataset

In [79]:
from pyspark.sql.functions import explode, array, lit, col, struct, desc, concat
from pyspark.sql.types import StringType

# Step 1: Get all column names from the DataFrame
columns = raw_dataset.columns

# Step 2: Create an array of structs with the column name and its content
exploded_df = raw_dataset.select(
    explode(
        array([
            struct(
                lit(column).alias("topic"),
                col(f"`{column}`.title").alias("title"),
                col(f"`{column}`.intro").alias("intro"),
                col(f"`{column}`.translated_intro").alias("translated_intro")
            )
            for column in columns
        ])
    ).alias("exploded")
)

# Step 3: Extract the fields from the struct and add the prompt column
transformed_df = exploded_df.select(
    col("exploded.title").alias("title"),
    lit("Translate this intro to english: ").alias("prompt"),
    col("exploded.intro").alias("intro"),
    col("exploded.translated_intro").alias("translated_intro")
)

# Step 4: Drop rows where any value is null
transformed_df = transformed_df.dropna(how="any")

# Step 5: Sort by title in descending order (Z to A)
transformed_df = transformed_df.orderBy(desc("title"))

In [70]:
# Display the schema of the resulting DataFrame
transformed_df.printSchema()

# Show a sample of the resulting DataFrame
transformed_df.show(40, 50)

root
 |-- title: string (nullable = true)
 |-- prompt: string (nullable = false)
 |-- intro: string (nullable = true)
 |-- translated_intro: string (nullable = true)



I0000 00:00:1744841625.386354 1156645 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


+---------------------+---------------------------------+----------------------------------------------------------------------------+--------------------------------------------------+
|                title|                           prompt|                                                                       intro|                                  translated_intro|
+---------------------+---------------------------------+----------------------------------------------------------------------------+--------------------------------------------------+
|            Ättestupa|Translate this intro to english: |                          Еттеступа (швед. Ättestupa) — обряд убивання ст...|Ättestupa (Swedish: Ättestupa) is a ritual of k...|
|            Ättestupa|Translate this intro to english: |                          Ättestupa (även ättestörta eller ättestapul) är...|Pedigree (also genitalia or pedigree) is a trad...|
|            Ättestupa|Translate this intro to english: |             

#### Create Apache Iceberg table in catalog

In [ ]:
ICEBERG_CATALOG = "CATALOG"
ICEBERG_SCHEMA = "default"
ICERBERG_TABLE = "wikipedia_translated_docs"

In [82]:
dataset_schema = f"({', '.join([f'{field.name} {type_mapping.get(type(field.dataType), str(field.dataType))}' for field in transformed_df.schema.fields])})"
dataset_schema

'(title string, prompt string, intro string, translated_intro string)'

In [ ]:
transformed_df.write.format("iceberg").save(f"{ICEBERG_CATALOG}.{ICEBERG_SCHEMA}.{ICERBERG_TABLE}")

#### Read Apache Iceberg table

In [90]:
iceberg_df = spark.read.table(f"{ICEBERG_CATALOG}.{ICEBERG_SCHEMA}.{ICERBERG_TABLE}")
iceberg_df.show()

+--------------------+--------------------+----------------------------------+--------------------+
|               title|              prompt|                             intro|    translated_intro|
+--------------------+--------------------+----------------------------------+--------------------+
|           Ättestupa|Translate this in...|              Еттеступа (швед. ...|Ättestupa (Swedis...|
|           Ättestupa|Translate this in...|              Ättestupa est la ...|Ättestupa is the ...|
|           Ättestupa|Translate this in...|              Ättestupa (även ä...|Pedigree (also ge...|
|           Ättestupa|Translate this in...|              Аттеступа (швед. ...|Attestupa (Swedis...|
|           Ättestupa|Translate this in...|              Ättestupa (términ...|Ättestupa (Scandi...|
|Zumbo's Just Dess...|Translate this in...|   赞波的甜点大赛（英语：Zumbo'...|Zumbo's Just Dess...|
|         Zoë Kravitz|Translate this in...|              Зої Ізабелла Крав...|Zoë Isabella Krav...|
|         Z

I0000 00:00:1744842967.222738 1156653 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


#### Generate dataset for finetuning

In [95]:
from pyspark.sql.functions import concat, col

finetune_dataset = iceberg_df.select(
    concat(col("prompt"), col("intro")).alias("input_prompt"),
    col("translated_intro").alias("expected_model_output")
)

finetune_dataset.show(5, 100)

I0000 00:00:1744843154.616600 1156649 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                        input_prompt|                                                                               expected_model_output|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|Translate this intro to english: Еттеступа (швед. Ättestupa) — обряд убивання старих членів племе...|   Ättestupa (Swedish: Ättestupa) is a ritual of killing old members of the tribe among the Vikings.|
|Translate this intro to english: Ättestupa est la pratique mythique du géronticide à l'époque de ...|Ättestupa is the mythical practice of geronticide at the time of Nordic prehistory

In [ ]:
finetune_dataset_pandas = finetune_dataset.toPandas()

train_set = finetune_dataset_pandas.sample(frac=0.8, random_state=42) 
test_set = finetune_dataset_pandas.drop(train_set.index)

In [97]:
def generate_records(df: pd.DataFrame) -> List:
    
    records = []

    for index, row in df.iterrows():

        input_prompt = row['input_prompt']
        expected_model_output = row['expected_model_output']

        record = {
          "contents": [
            { "role": "user", "parts": [ { "text": input_prompt } ] },
            { "role": "model", "parts": [ { "text": expected_model_output } ] } ] 
        }

        records.append(record)
        
    return records

In [ ]:
train_records = generate_records(train_set)[:1000] # Select 1000 training records for fine tuning training
val_records = generate_records(test_set)[:300]     # Select 300 eval records for fine tuning evaluation

In [ ]:
TRAIN_FILE_NAME = "wikipedia_translated/records/fine-tuning-train-dataset.jsonl"
VAL_FILE_NAME = "wikipedia_translated/records/fine-tuning-val-dataset.jsonl"

In [110]:
def upload_gcs(records: List, file_name: str, bucket_name: str = BUCKET_NAME, project_id: str = PROJECT_ID) -> str:
    
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)

    jsonl_data = "\n".join(json.dumps(item) for item in records)
    blob.upload_from_string(jsonl_data)
    
    uri = f"gs://{bucket_name}/{file_name}"

    return uri

In [111]:
uri_train = upload_gcs(train_records, TRAIN_FILE_NAME)
uri_val = upload_gcs(val_records, VAL_FILE_NAME)

#### Run finetuning job on Vertex AI

In [112]:
from google import genai
from google.genai import types

In [ ]:
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=REGION
)

In [104]:
GEMINI_MODEL = "gemini-2.0-flash-001"

In [ ]:
tuned_model_name = "tuned_gemini_translation"

tuning_job = client.tunings.tune(
    base_model = GEMINI_MODEL,
    training_dataset = types.TuningDataset(gcs_uri = uri_train),
    config=types.CreateTuningJobConfig(
        epoch_count= 8,
        tuned_model_display_name=tuned_model_name,
        adapter_size = "ADAPTER_SIZE_FOUR",
        learning_rate_multiplier = 0.8,
        validation_dataset = types.TuningDataset(gcs_uri=uri_val)
    )
)

In [ ]:
while not tuning_job.has_ended:
    time.sleep(30)
    tuning_job = client.tunings.get(name=tuning_job.name)

In [ ]:
print(tuning_job.tuned_model.model)
print(tuning_job.tuned_model.endpoint)
print(tuning_job.experiment)

In [126]:
def predict(prompt, model):

    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt)
            ]
        )
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature = 0.5,
        max_output_tokens = 128,
        response_mime_type = "text/plain",
        safety_settings = [types.SafetySetting(
            category = 'HARM_CATEGORY_UNSPECIFIED',
            threshold = 'BLOCK_ONLY_HIGH',
        )]
    )
    
    response = client.models.generate_content(model=model,
                                                contents=contents,
                                                config=generate_content_config)
    return response.text

In [124]:
input_prompt = """Translate this intro to english: You (Estilizado como YOU - Sendo nomeado no Brasil como Você, em Portugal como Tu) é uma série de televisão americana de suspense psicológico desenvolvida por Greg Berlanti e Sera Gamble.
Produzido pela Warner Horizon Television, em associação com Alloy Entertainment e A&E Studios. A série é baseada no romance de 2014 de mesmo nome de Caroline Kepnes.
A primeira temporada segue Joe Goldberg, gerente de uma livraria de Nova York e um serial killer que se apaixona por uma cliente chamada Guinevere Beck e rapidamente desenvolve uma obsessão extrema, tóxica e delirante.
A segunda temporada segue Joe enquanto ele se muda para Los Angeles e se apaixona por Love Quinn, chef e sócia de uma rede de produtos naturais.
A primeira temporada, que foi lançada em 2018, é estrelada por Penn Badgley, Elizabeth Lail, Luca Padovan, Zach Cherry e Shay Mitchell.
Para a segunda temporada, Ambyr Childers foi promovida a regular na série, juntando-se aos recém-escalados Victoria Pedretti, James Scully, Jenna Ortega e Carmela Zumbado.
A série estreou na Lifetime em 9 de setembro de 2018 nos Estados Unidos e transmitida internacionalmente pela Netflix em 26 de dezembro de 2018.
A série atraiu um público limitado na Lifetime antes de se tornar mais popular e um sucesso crítico para a Netflix, com mais de 43 milhões de espectadores tendo transmitido a primeira temporada após sua estreia no serviço de streaming.
A Lifetime anunciou que You foi renovada para uma segunda temporada baseada no romance seguinte de Kepnes, Hidden Bodies, em 26 de julho de 2018, antes da estreia da série.
Em dezembro de 2018, foi anunciado que a série mudaria para a Netflix como um título Original Netflix.
A segunda temporada foi lançada exclusivamente na Netflix em 26 de dezembro de 2019.
Em janeiro de 2020, a série foi renovada para uma terceira temporada pela Netflix, que conta com Badgley e Pedretti reprisando seus papéis.
No dia 30 de agosto de 2021, foi confirmado que a terceira temporada irá estrear dia 15 de outubro de 2021.
Em outubro de 2021, antes da estreia da terceira temporada, a série foi renovada para uma quarta temporada."""

In [127]:
predict(input_prompt, tuning_job.tuned_model.endpoint)

INFO:root:AFC is enabled with max remote calls: 10.
INFO:root:AFC remote call 1 is done.


'You (Stylized as YOU - Named in Brazil as Você, in Portugal as Tu) is an American psychological thriller television series developed by Greg Berlanti and Sera Gamble.\nProduced by Warner Horizon Television, in association with Alloy Entertainment and A&E Studios. The series is based on the 2014 novel of the same name by Caroline Kepnes.\nThe first season follows Joe Goldberg, manager of a New York bookstore and serial killer who falls in love with a customer named Guinevere Beck and quickly develops an extreme, toxic and delusional obsession.\nThe second season follows Joe as he moves to Los Angeles and falls in'